In [15]:
from os import path, getenv
from mlrun import new_project, mlconf

#project_name = '-'.join(filter(None, ['getting-started-iris', getenv('V3IO_USERNAME', None)]))
project_name = "parquez"
project_path = path.abspath('./')
project = new_project(project_name, project_path)
project.save()
print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/parquez
Project name: parquez


In [16]:
out = mlconf.artifact_path or path.abspath('./data')
# {{run.uid}} will be substituted with the run id, so output will be written to different directoried per run
artifact_path = path.join(out, '{{run.uid}}')

In [17]:
%env PYTHONPATH=./

env: PYTHONPATH=./


In [18]:
project.set_function("create_crontab.py", 'crontab', kind='job', image='aviaigz/parquez')

In [19]:
from mlrun import run_local, mount_v3io

In [20]:
project.func('crontab').apply(mount_v3io())
project.func('crontab').set_env('PYTHONPATH', project_path)
project.func('crontab').spec.artifact_path = 'User/artifacts'
project.func('crontab').spec.service_account='mlrun-api'

In [ ]:
artifact_path = '/User/artifacts'
# #project.func('parquezrun').run()
project.func('crontab').run( params = {'view_name':'view_name'
         ,'partition_by':'h'
         ,'partition_interval':'1h'
         ,'real_time_window':'3h'
         ,'historical_retention':'7d'
         ,'real_time_table_name':'faker'
         ,'config_path':'/User/parquez/config/parquez.ini'},artifact_path=artifact_path)

[mlrun] 2020-07-07 11:58:27,463 starting run crontab uid=80b3d6f1307d440aad67bcd61850f264  -> http://10.199.81.213:8080
[mlrun] 2020-07-07 11:58:27,575 Job is running in the background, pod: crontab-fnm2f
[mlrun] 2020-07-07 11:58:34,026 warning!, server (0.4.7) and client (0.4.9) ver dont match
[mlrun] 2020-07-07 11:58:34,039 starting local run: main.py # main
